In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../../data/raw/news/newsapiorg_labelled.csv", index_col=0)
df.head()

,title,publishedAt,url,content,vader_neg,vader_neu,vader_pos,vader_comp,vader_label,label,include,relevant
Column1,,,,,,,,,,,,
1,Key remarks from Russian c.bank governor after...,2022-02-11T12:26:00Z,https://www.reuters.com/markets/currencies/key...,"MOSCOW, Feb 11 (Reuters) - The Russian Central...",0.000,1.000,0.0,0.0000,0,0,1,0
2,LIVE MARKETS Forex: worrying about an inverted...,2022-02-18T12:33:00Z,https://www.reuters.com/markets/stocks/live-ma...,Feb 18 - Welcome to the home for real-time cov...,0.211,0.789,0.0,-0.3400,-1,-1,1,1
3,Dollar takes a break from this week's slide ah...,2022-02-03T02:18:00Z,https://www.reuters.com/markets/europe/dollar-...,"SINGAPORE, Feb 3 (Reuters) - The dollar found ...",0.000,1.000,0.0,0.0000,0,-1,1,1
4,Euro bounce pauses ahead of U.S. inflation - R...,2022-02-08T01:16:00Z,https://www.reuters.com/business/euro-bounce-p...,"SINGAPORE, Feb 8 (Reuters) - A resurgent euro ...",0.000,1.000,0.0,0.0000,0,1,1,1
5,Dollar near two-week high amid jitters over ha...,2022-01-25T01:52:00Z,https://www.reuters.com/markets/europe/dollar-...,"TOKYO, Jan 25 (Reuters) - The safe-haven U.S. ...",0.197,0.803,0.0,-0.4019,-1,-1,1,1


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.title, df.label, test_size = 0.2)
